# **Global Needs**

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
leRate = 3e-5
baSize = 16

# **IndoBert Lite (without KD)**

In [ ]:
!pip install datasets

from transformers import AutoModelForSequenceClassification
from datasets import Dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
# load lite model
lite_model = AutoModelForSequenceClassification.from_pretrained('indobenchmark/indobert-lite-base-p2', num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-lite-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import json
# load tokenized data for lite model (only train and validation)
with open('lite_tokenized_data.json', 'r') as f:
    lite_tokenized_data = json.load(f)

train_dataset_lite = Dataset.from_dict(lite_tokenized_data['train'])
val_dataset_lite = Dataset.from_dict(lite_tokenized_data['val'])

In [ ]:
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss

def collate_fn(batch):
    return {
        'input_ids': torch.stack([torch.tensor(item['input_ids']) for item in batch]),
        'attention_mask': torch.stack([torch.tensor(item['attention_mask']) for item in batch]),
        'labels': torch.tensor([item['labels'] for item in batch], dtype=torch.long)
    }

# DataLoader for train dataset with collate_fn
train_loader = DataLoader(
    train_dataset_lite,
    batch_size=baSize,
    shuffle=True,
    collate_fn=collate_fn
)
print(train_loader)

# DataLoader for validation dataset
val_loader = DataLoader(
    val_dataset_lite,
    batch_size=baSize,
    shuffle=False,  # no need to shuffle for validation
    collate_fn=collate_fn
)
print(val_loader)


# Define loss function with weighted loss
criterion = CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(lite_model.parameters(), lr=leRate)

In [ ]:
from sklearn.metrics import classification_report
# Early Stopping Params
best_val_loss = float('inf')
patience = 3
min_delta = 0.001
patience_counter = 0
best_baseline_model = None

lite_model.to(device)

# Training loop
for epoch in range(50):
    total_train_loss = 0
    lite_model.train()

    # Training phase
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = lite_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Compute loss
        loss = criterion(logits, labels)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    # Validation phase
    lite_model.eval()
    total_val_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = lite_model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Compute validation loss
            loss = criterion(logits, labels)
            total_val_loss += loss.item()

            # Collect predictions and true labels for evaluation
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate average losses and metrics
    avg_train_loss = total_train_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(val_loader)

    print(f"Epoch {epoch + 1}")
    print(f"Training Loss: {avg_train_loss:.4f}")
    print(f"Validation Loss: {avg_val_loss:.4f}")
    print("Validation Metrics:")
    print(classification_report(all_labels, all_preds, target_names=['Positive', 'Negative']))

    # Early Stopping
    if avg_val_loss < best_val_loss - min_delta:
        best_val_loss = avg_val_loss
        patience_counter = 0
        # Save the best model
        save_dir = f"best_baseline_model_epoch_{epoch + 1}"
        lite_model.save_pretrained(save_dir)  # Save the model checkpoint
        best_baseline_model = lite_model  # Update the best model reference
    else:
        patience_counter += 1
        print(f"No improvement in validation loss for {patience_counter} epoch(s).")

        if patience_counter >= patience:
            print("Early stopping triggered.")
            break


model.safetensors:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

Epoch 1
Training Loss: 0.3233
Validation Loss: 0.3240
Validation Metrics:
              precision    recall  f1-score   support

    Positive       0.75      0.66      0.70       349
    Negative       0.90      0.94      0.92      1191

    accuracy                           0.87      1540
   macro avg       0.83      0.80      0.81      1540
weighted avg       0.87      0.87      0.87      1540

Epoch 2
Training Loss: 0.2350
Validation Loss: 0.2957
Validation Metrics:
              precision    recall  f1-score   support

    Positive       0.78      0.60      0.68       349
    Negative       0.89      0.95      0.92      1191

    accuracy                           0.87      1540
   macro avg       0.83      0.78      0.80      1540
weighted avg       0.87      0.87      0.87      1540

Epoch 3
Training Loss: 0.1605
Validation Loss: 0.3051
Validation Metrics:
              precision    recall  f1-score   support

    Positive       0.78      0.63      0.70       349
    Negative   

In [ ]:
def evaluate_model(model, dataset, collate_fn):
    model.to(device)
    model.eval()
    data_loader = DataLoader(dataset, batch_size=baSize, collate_fn=collate_fn)
    all_preds, all_labels = [], []

    for batch in data_loader:
        input_ids = torch.tensor(batch['input_ids']).to(device)
        attention_mask = torch.tensor(batch['attention_mask']).to(device)
        labels = torch.tensor(batch['labels']).to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return classification_report(all_labels, all_preds, target_names=['Positive', 'Negative'])

In [ ]:
# load tokenized data for lite model (only test)
with open('lite_tokenized_data.json', 'r') as f:
    lite_tokenized_data = json.load(f)

test_dataset_lite = Dataset.from_dict(lite_tokenized_data['test'])

In [ ]:
lite_without_KD = best_baseline_model

In [ ]:
# Evaluasi lite no KD Model pada Testing Set
print("Evaluasi lite no KD  Model (Testing Set):")
print(evaluate_model(lite_without_KD, test_dataset_lite, collate_fn))

Evaluasi lite no KD  Model (Testing Set):


<ipython-input-11-b4274105a7a2>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch['input_ids']).to(device)
<ipython-input-11-b4274105a7a2>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch['attention_mask']).to(device)
<ipython-input-11-b4274105a7a2>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['labels']).to(device)


              precision    recall  f1-score   support

    Positive       0.78      0.59      0.67       340
    Negative       0.89      0.95      0.92      1200

    accuracy                           0.87      1540
   macro avg       0.84      0.77      0.80      1540
weighted avg       0.87      0.87      0.87      1540

